<img src="classical_gates.png" />

In [ ]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer

from qiskit.tools.jupyter import *
from qiskit.visualization import *

In [ ]:
qr = QuantumRegister(2, 'qubit')
cr = ClassicalRegister(2, name="bit")
circuit = QuantumCircuit(qr, cr)
circuit.h(qr[0])
circuit.cx(qr[0], qr[1])
circuit.measure(qr, cr)
circuit.draw('mpl', initial_state=True)

In [ ]:
# Load simulator
local_simulator = BasicAer.get_backend('qasm_simulator')

job = execute(circuit, backend=local_simulator, shots=1000)
job.result().get_counts()

# Deutsch Algorithm

<table>
<tbody>
<tr>
<td colspan="2"><img src="deutsch_problem.png" /></td>
</tr>
<tr>
<td><img src="classic_oracle.png" /></td>
<td><img src="quantum_oracle.png" /></td>
</tr>
</tbody>
</table>

## Oracle 4: Constant zero

In [ ]:
input = QuantumRegister(1, name='input')
output = QuantumRegister(1, name='output')
constant0 = QuantumCircuit(input, output, name='oracle')
oracle4 = constant0.to_instruction()
constant0.draw('mpl', initial_state=True)

## Oracle 3: Constant one

In [ ]:
input = QuantumRegister(1, name='input')
temp = QuantumRegister(1, name='output')
constant1 = QuantumCircuit(input, temp, name='oracle')
constant1.x(temp)
oracle3 = constant1.to_instruction()
constant1.draw('mpl', initial_state=True)

## Oracle 1: Identity

In [ ]:
input = QuantumRegister(1, name='input')
temp = QuantumRegister(1, name='output')
identity = QuantumCircuit(input, temp, name='oracle')
identity.cx(input, temp)
oracle1 = identity.to_instruction()
identity.draw('mpl', initial_state=True)

## Oracle 2: Invert

In [ ]:
input = QuantumRegister(1, name='input')
output = QuantumRegister(1, name='output')
invert = QuantumCircuit(input, output, name='oracle')
invert.cx(input, output)
invert.x(output)
oracle2 = identity.to_instruction()
invert.draw('mpl', initial_state=True)

## Run an oracle

In [ ]:
result = ClassicalRegister(1, name='result')
circuit = QuantumCircuit(input, output, result)
circuit.x(input) # <- set input to 1
circuit.barrier()
circuit += identity
circuit.barrier()
circuit.measure(output, result)
circuit.draw('mpl', initial_state=True)

In [ ]:
execute(circuit, backend=local_simulator, shots=10000).result().get_counts()

## Running Deutsch's Algorithm

In [ ]:
qr = QuantumRegister(2, name='qubits')
cr = ClassicalRegister(1, name='result')
circuit = QuantumCircuit(qr, cr)
circuit.x(qr[1])
circuit.h(qr)
circuit.append(oracle3, [qr[0], qr[1]])
circuit.h(qr[0])
circuit.measure(qr[0], cr[0]);
circuit.draw('mpl', initial_state=True)

In [ ]:
counts = execute(circuit, backend=local_simulator, shots=1).result().get_counts()
counts['BALANCED'] = counts.pop('1', None)
counts['CONSTANT'] = counts.pop('0', None)
print(counts)

# Real device!

In [ ]:
import qiskit.tools.jupyter
from qiskit import IBMQ
from qiskit.providers.ibmq import least_busy

In [ ]:
provider = IBMQ.load_account()
least_busy_device = least_busy(provider.backends(simulator=False, filters=lambda b: b.configuration().n_qubits >= 2))
least_busy_device

In [ ]:
job = execute(circuit, backend=least_busy_device, shots=1000)

In [ ]:
print(job.status())

In [ ]:
counts = job.result().get_counts()
print(counts)
counts['BALANCED'] = counts.pop('1', None)
counts['CONSTANT'] = counts.pop('0', None)
print(counts)